In [1]:
import os
import csv
import pandas as pd
import numpy as np
from IPython.display import clear_output
import ast
import pymongo as pm
from datetime import datetime


### Load connection to database

In [2]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the stanford-cogsci.org user
pswd = auth.values[0][0]
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')

In [3]:
db = conn['devphotodraw_recognition']
coll = db['batched_12afc']

In [4]:
# db = conn['kiddraw']
# coll = db['check_invalid_v5_dev']
# coll.count()

In [5]:
analysis_dir = os.getcwd()
output_dir = os.path.join(analysis_dir,'csv_out')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Only prolific workers have id's that are strings (parsed from url)

In [24]:
catch

In [26]:
catch_responses = []
catch = coll.find({'sub_id': {'$type' : "string"}, 'version': this_version, 'dataType': 'catch_trial' })
for c in catch:
   catch_responses.append(c['catch'])

catch_responses

[u'Describing illustrations from a list of choices',
 u'Guess what people intended to draw from a list of choices.',
 u'Recognizing drawings',
 u'Identifying drawings from an available set of answers',
 u'Looking at drawings and guessing what they were from a set group of choices. ',
 u'I was looking at drawings and guessing what they were from a list of words.',
 u'naming the images that shows with the given options',
 u'Identifying objects.',
 u'Identifying drawings; matching what we believe was being drawn from a list of options',
 u'Naming drawings']

In [6]:
this_version = 'batch5_production_june2021'
# prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': 'batch1_production_june2021' }).distinct('sub_id')
prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': this_version }).distinct('sub_id')
np.size(prolific_workers)
prolific_workers

[u'}}&STUDY_ID={{%STUDY_ID',
 u'60ca70f0bb3fef9c094682e',
 u'5e342cecde69fe3e0dd20d4',
 u'60c918a1b8ecff26d3de9df',
 u'60c70a25e730952df844d1e',
 u'60ca71e1a0157b6cce37959',
 u'602be5d586b0773c600c4ff',
 u'60ca42c682bc800df005667',
 u'60c6d966de0036fe3659e68',
 u'60ca6e4fd5183359ae3255a',
 u'60c940f718d8d7c39ed570d']

In [9]:
prolific_workers.pop()

###### Open up variables for CSV writing
# basic descriptors
subID = []; 
imageName = []
guessed_category = []
drawn_category = []
version = []

In [11]:
del prolific_workers[0]


In [12]:
prolific_workers

[u'60ca70f0bb3fef9c094682e',
 u'5e342cecde69fe3e0dd20d4',
 u'60c918a1b8ecff26d3de9df',
 u'60c70a25e730952df844d1e',
 u'60ca71e1a0157b6cce37959',
 u'602be5d586b0773c600c4ff',
 u'60ca42c682bc800df005667',
 u'60c6d966de0036fe3659e68',
 u'60ca6e4fd5183359ae3255a']

In [13]:
for w in prolific_workers:
    print w
    entries = coll.find({'$and': [{'sub_id': w, 'dataType': 'recognition_rating'}]})
    num_trials =  entries.count()
    print num_trials
    for e in entries:
        subID.append(e['sub_id'])
        imageName.append(e['imageName'])
        guessed_category.append(e['guessed_category'])
        drawn_category.append(e['imageCategory'])
        version.append(e['version'])

60ca70f0bb3fef9c094682e
144
5e342cecde69fe3e0dd20d4
144
60c918a1b8ecff26d3de9df
142
60c70a25e730952df844d1e
138
60ca71e1a0157b6cce37959
144
602be5d586b0773c600c4ff
144
60ca42c682bc800df005667
144
60c6d966de0036fe3659e68
144
60ca6e4fd5183359ae3255a
144


In [27]:

## write out csv 
X_out = pd.DataFrame([subID, imageName, guessed_category, drawn_category, version])
X_out = X_out.transpose()
X_out.columns = ['subID','imageName', 'guessed_category','drawn_category','version']
X_out.to_csv(os.path.join(output_dir, this_version + '.csv'))


In [62]:
# path = 'P_bird_sketch_age5_IPAD5_THU5F3.png'
# path = 'P_airplane_sketch_age4_CDM_photodraw_e21547242197593.png'
# condition = path.split('/')[-1].split('_')[4]
# if path.find('CDM')!=-1:
#     site = 'CDM'
# else:
#     site = 'THU'
# site

'CDM'

In [ ]:
# mturk_id
# assignment_id
# trial_id
# hit_id